In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torch.nn.utils.rnn as rnn_utils

import torch.nn as nn

from pytorch_metric_learning import losses as ml_losses 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import os
import gc
import csv

import json
# from tqdm.auto import tqdm
from tqdm import tqdm
from datetime import datetime, date, time, timezone, timedelta
import pickle
import random

import math
from scipy.spatial import distance_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sn
from IPython.display import clear_output
from ipywidgets import Output
from IPython import display

In [2]:
#device = torch.cuda.device(2)
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [3]:
seed_value = 17
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
random.seed(17)

In [4]:
target_players = ['Anders ANTONSEN',
 'Anthony Sinisuka GINTING',
 'CHEN Long',
 
 'CHOU Tien Chen',
 'Jonatan CHRISTIE',
 'Kento MOMOTA',
 'Khosit PHETPRADAB',
 'NG Ka Long Angus',
 
 'SHI Yuqi',
 
 'Viktor AXELSEN',
 'WANG Tzu Wei']

In [5]:
def player2cat(player):
    p2c = {'Anders ANTONSEN': 0, 'Anthony Sinisuka GINTING': 1, 'CHEN Long': 2,
      'CHOU Tien Chen': 3, 'Jonatan CHRISTIE': 4, 'Kento MOMOTA': 5,
     'Khosit PHETPRADAB': 6, 'NG Ka Long Angus': 7,
     'SHI Yuqi': 8,  'Viktor AXELSEN': 9, 'WANG Tzu Wei': 10}
    return p2c[player]

def cat2player(cat):
    c2p = {0:'ANTONSEN', 1:'GINTING', 2:'Long',  3:'CHOU', 4:'CHRISTIE',
           5:'MOMOTA', 6:'PHETPRADAB', 7:'NG',  8:'SHI', 
           9:'AXELSEN', 10:'WANG'}
    return c2p[cat]

def generate_labels(rally_data):
    # predict player A and B
    playerA = rally_data['name_A'].values[0]
    playerB = rally_data['name_B'].values[0]

    if playerA in target_players and playerB in target_players:
        return np.array([player2cat(playerA)]),  np.array([player2cat(playerB)])
    elif playerA not in target_players and playerB in target_players:
        return None,  np.array([player2cat(playerB)])
    elif playerA in target_players and playerB not in target_players:
        return np.array([player2cat(playerA)]),  None
    elif playerA in target_players and playerB in target_players:
        return None,  None

In [6]:
def type2cat(shot_type):
    t2c = {'發短球': 0, '長球': 1, '推撲球': 2, '殺球': 3, '接殺防守': 4, '平球': 5,
           '網前球': 6, '挑球': 7, '切球': 8, '發長球': 9}
    return t2c[shot_type]

def process_rally(rally_data):
    ## process config
    mean_x, std_x = 630., 160.
    mean_y, std_y = 470., 105.
    
    drop_cols = ['rally', 'match_id', 'set', 'rally_id', 'ball_round', 'time', 'frame_num', 'db', 'flaw', 'lose_reason', 'win_reason', 'type', 'server', # no need
                 'hit_area', 'landing_area', 'player_location_area', 'opponent_location_area', # area dup with x/y
                 'name_A', 'name_B', 'getpoint_player', 'roundscore_A', 'roundscore_B', # rally-wise features, maybe use later
                 'landing_height', 'landing_x', 'landing_y'] # landing info is dup with hitting
    
    ## Get player name for checking
    playerA = rally_data['name_A'].values[0]
    playerB = rally_data['name_B'].values[0]    
    
    ## process frame_num (time), get frame difference between last shot and this shot, 0 if serve ball 
    frame_diff = np.pad(rally_data['frame_num'].values[1:] - rally_data['frame_num'].values[:-1], (1, 0), mode='constant')
    rally_data['frame_diff'] = frame_diff
    
    ## NaN convert to binary
    rally_data['aroundhead'] = (rally_data['aroundhead'] == 1).astype(int)
    rally_data['backhand'] = (rally_data['backhand'] == 1).astype(int)
    
    ## Player A/B, convert to binary
    rally_data['player'] = (rally_data['player'] == 'A').astype(int)
    
    ## height convert to binary
    rally_data['hit_height'] = (rally_data['hit_height'] -1)
    rally_data['landing_height'] = (rally_data['landing_height'] -1)
    
    ## hit_x, hit_y fill with player location
    rally_data['hit_x'].values[0] = rally_data['player_location_x'].values[0]
    rally_data['hit_y'].values[0] = rally_data['player_location_y'].values[0]
    
    ## x/y standardization
    rally_data['hit_x'] = (rally_data['hit_x'] - mean_x)/std_x
    rally_data['hit_y'] = (rally_data['hit_y'] - mean_y)/std_y
    rally_data['landing_x'] = (rally_data['landing_x'] - mean_x)/std_x
    rally_data['landing_y'] = (rally_data['landing_y'] - mean_y)/std_y
    rally_data['player_location_x'] = (rally_data['player_location_x'] - mean_x)/std_x
    rally_data['player_location_y'] = (rally_data['player_location_y'] - mean_y)/std_y
    rally_data['opponent_location_x'] = (rally_data['opponent_location_x'] - mean_x)/std_x
    rally_data['opponent_location_y'] = (rally_data['opponent_location_y'] - mean_y)/std_y
    
    # type convert to category
    rally_data['type_code'] = [type2cat(t) for t in rally_data['type'].values]
    
    ## drop unneccesary columns
    rally_data.drop(columns=drop_cols, inplace=True)
    
    ## create a copy of the rally but with opposite player 
    inverse = rally_data.copy()
    inverse['player'] = (inverse['player']+1)%2
    
    if playerA in target_players and playerB in target_players:
        return rally_data.values, inverse.values
    elif playerA not in target_players and playerB in target_players:
        return None, inverse.values
    elif playerA in target_players and playerB not in target_players:
        return rally_data.values, None
    elif playerA in target_players and playerB in target_players:
        return None,  None


In [7]:
def collate_fn(data):
    seq, label = zip(*data)
    seq = list(seq)
    label = list(label)
    pairs = [(s, l) for s, l in zip(seq, label)]
    pairs.sort(key=lambda x: len(x[0]), reverse=True)
    seq = [s for s, l in pairs]
    label = [l for s, l in pairs]
    seq_length = [len(sq) for sq in seq]
    seq = rnn_utils.pad_sequence(seq, batch_first=True, padding_value=0)
    labels = torch.zeros(0, 1)
    for l in label:
        labels = torch.cat([labels, l], axis=0)
    return seq, seq_length, labels

In [8]:
data_path = 'aug_set.csv'

In [9]:
def check_nan(np_rally):
    if np_rally is None:
        return False
    else:
        return np.isnan(np.sum(np_rally))

In [10]:
data = pd.read_csv(data_path)

rids = set()
# drop flawed rallies, record label distribution
for rally in tqdm(data['rally_id'].unique()):
    if data.loc[data['rally_id']==rally]['flaw'].any() or len(data.loc[data['rally_id']==rally])<=2 or rally in [578, 596]:
        continue
    else:
        rids.add(rally)

100%|██████████| 2575/2575 [00:02<00:00, 1242.45it/s]


In [11]:
def train_test_split(label2rids, test_ratio):
    test = random.sample(label2rids, k=round(len(label2rids)*test_ratio))
    train = [rid for rid in label2rids if rid not in test]
    return train, test

In [12]:
class PlayerClassificationDataset(Dataset):
    def __init__(self, data, rids, split):
        self.data = data
        self.rids = rids
        self.seqs = []
        self.labels = []
        
        pbar = tqdm(rids)
        pbar.set_description('Processing %s rally data'%split)
        tmp = [process_rally(self.data.loc[self.data['rally_id']==rally].copy()) for rally in pbar]
        for seq1, seq2 in tmp:
            if seq1 is not None:
                self.seqs.append(seq1)
            if seq2 is not None:
                self.seqs.append(seq2)
        
        pbar2 = tqdm(rids)
        pbar2.set_description('Generating %s labels'%split)
        tmp = [generate_labels(self.data.loc[self.data['rally_id']==rally].copy()) for rally in pbar2]
        for label1, label2 in tmp:
            if label1 is not None:
                self.labels.append(label1)
            if label2 is not None:
                self.labels.append(label2)        
        
        # checking data are clear, remove those with NaN
        self.nan_checking()
    def __len__(self):
        return len(self.seqs)
    
    def __getitem__(self, index):
        return torch.Tensor(self.seqs[index]), torch.Tensor(self.labels[index]).unsqueeze(0)
    
    def nan_checking(self):
        bad_idxs = [idx for idx in range(len(self.seqs)) if check_nan(self.seqs[idx])]
        self.seqs = [seq for idx, seq in enumerate(self.seqs) if idx not in bad_idxs]
        self.labels = [label for idx, label in enumerate(self.labels) if idx not in bad_idxs]
        print("Removed %d rallies with NaN value!"%len(bad_idxs))

In [13]:
class SubsequenceClassificationDataset(Dataset):
    def __init__(self, data, rids, split, p, threshold):
        self.data = data
        self.rids = rids
        self.seqs = []
        self.labels = []
        self.p = p
        self.threshold = threshold
        
        pbar = tqdm(rids)
        pbar.set_description('Processing %s rally data'%split)
        tmp = [process_rally(self.data.loc[self.data['rally_id']==rally].copy()) for rally in pbar]
        for seq1, seq2 in tmp:
            if seq1 is not None:
                self.seqs.append(seq1)
            if seq2 is not None:
                self.seqs.append(seq2)
        
        pbar2 = tqdm(rids)
        pbar2.set_description('Generating %s labels'%split)
        tmp = [generate_labels(self.data.loc[self.data['rally_id']==rally].copy()) for rally in pbar2]
        for label1, label2 in tmp:
            if label1 is not None:
                self.labels.append(label1)
            if label2 is not None:
                self.labels.append(label2)        
        
        # checking data are clear, remove those with NaN
        self.nan_checking()
    def __len__(self):
        return len(self.seqs)
    
    def __getitem__(self, index):
        ### if sequence length is longer than a threshold, random sample a sub-sequence with probability p
        dice = random.random()
        seq = self.seqs[index]
        if dice < self.p and len(seq) >= self.threshold:
            sub_len = random.randint(5, len(seq))
            start = random.randint(0, len(seq) - sub_len)
            sub_seq = seq[start:start+sub_len]
            return torch.Tensor(sub_seq), torch.Tensor(self.labels[index]).unsqueeze(0)
        else:
            return torch.Tensor(seq), torch.Tensor(self.labels[index]).unsqueeze(0)
    
    def nan_checking(self):
        bad_idxs = [idx for idx in range(len(self.seqs)) if check_nan(self.seqs[idx])]
        self.seqs = [seq for idx, seq in enumerate(self.seqs) if idx not in bad_idxs]
        self.labels = [label for idx, label in enumerate(self.labels) if idx not in bad_idxs]
        print("Removed %d rallies with NaN value!"%len(bad_idxs))

In [14]:
class TripletSamplingDataset(Dataset):
    def __init__(self, dataset, num_classes, weights=None):
        self.num_classes = num_classes
        self.dataset = dataset
        self.num_seqs = len(self.dataset)
        if weights is not None:
            self.weights = weights
        else:
            # use uniform if weights not provided
            self.weights = np.array([1 for i in range(self.num_classes)])
        
        self.label2idxs = {i: [] for i in range(num_classes)}
        self.build_label2idxs()
        
        self.dist_mtrx = np.random.rand(self.num_seqs, self.num_seqs)
    
    def __len__(self):
        return self.num_seqs
    
    def __getitem__(self, index):
        anchor, anchor_label = self.dataset[index]
        positive, positive_label = self.dataset[self.sample_positive(index, anchor_label.long().item())]
        negative, negative_label = self.dataset[self.sample_negative(index, anchor_label.long().item())]
        return anchor, positive, negative, anchor_label, positive_label, negative_label
    
    def build_label2idxs(self):
        for idx, (seq, label) in enumerate(self.dataset):
            label = label.long().item()
            self.label2idxs[label].append(idx)
    
    def sample_positive(self, anchor_index, label):
        pool = self.label2idxs[label].copy()
        pool.remove(anchor_index)
        return random.choice(pool)
    
    def sample_negative(self, anchor_index, label):
        # First sample the negative class according to given class weight
        # Then perform distance weighted sampling on the target class's samples
        class_pool = np.array([i for i in range(self.num_classes) if i!=label])
        weights = self.weights[class_pool]
        sample_class = np.random.choice(class_pool, size=1, p=weights/weights.sum())[0].astype(int)
        pool = self.label2idxs[sample_class].copy()
        # do sorting to prevent potential problem, might cause efficiency problem though
        pool.sort()
        # pool is a list of idx, so we now get distance of these idx to anchor
        dist = self.dist_mtrx[anchor_index][pool]
        # uniformly sample a distance in [dist.min(), dist.max()]
        sampled_dist = np.random.uniform(dist.min(), dist.max())
        # get the index which have closest distance to the sampled distance
        closest_idx = np.argmin(np.power(dist - sampled_dist, 2))
        return pool[closest_idx]
    
    def distance_weighted_negative_sample(self, anchor_index, label):
        class_pool = np.array([i for i in range(self.num_classes) if i!=label])
        index_pool = []
        for label in class_pool:
            index_pool.extend(self.label2idxs[label].copy())
        index_pool.sort()
        dist_list = self.dist_mtrx[anchor_index][index_pool]
        dist_list = [dist if dist < 1e5 else 1e5 for dist in dist_list]
        dist_list = [dist if dist != 0 else 1e-5 for dist in dist_list]
        dist_list = np.array(dist_list)
        dist_list = 1 / dist_list
        sample_idx = np.random.choice(index_pool, size=1, p=dist_list/dist_list.sum())[0].astype(int)
        return sample_idx
    
    def update_distance(self, embeddings):
        self.dist_mtrx = torch.cdist(embeddings, embeddings, p=2.0, compute_mode='use_mm_for_euclid_dist_if_necessary').cpu().numpy()

In [15]:
def triplet_collate(data):
    anchor, positive, negative, anchor_label, positive_label, negative_label = zip(*data)
    anchor, positive, negative = list(anchor), list(positive), list(negative)
    anchor_label, positive_label, negative_label = list(anchor_label), list(positive_label), list(negative_label)

    anchor_label = torch.tensor(anchor_label)
    anchor_len = [len(a) for a in anchor]
    anchor = rnn_utils.pad_sequence(anchor, batch_first=True, padding_value=0)


    positive_label = torch.tensor(positive_label)
    positive_len = [len(p) for p in positive]
    positive = rnn_utils.pad_sequence(positive, batch_first=True, padding_value=0)

    negative_label = torch.tensor(negative_label)
    negative_len = [len(n) for n in negative]
    negative = rnn_utils.pad_sequence(negative, batch_first=True, padding_value=0)

    return anchor, positive, negative, anchor_len, positive_len, negative_len, anchor_label

In [16]:
class CNNRNN(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, out_dim, GRU_layers):
        super(CNNRNN, self).__init__()
        self.input_dim = input_dim
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.GRU_layers = GRU_layers
        
        self.type_embedding = nn.Embedding(10, self.embed_dim)
        self.proj = nn.Linear(self.input_dim - 1, self.hidden_dim - self.embed_dim)
        self.fc1 = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.conv1 = nn.Conv1d(self.hidden_dim, self.hidden_dim, kernel_size=3, stride=1, padding=1)
        self.GRU = nn.GRU(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.GRU_layers, bias=True, batch_first=True, bidirectional=True)
        #self.output = MLP(self.hidden_dim, self.hidden_dim//16, self.out_dim)
        self.fc2 = nn.Linear(self.hidden_dim*2, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.out_dim) # in_dim is hidden_dim
        self.relu = nn.ReLU()
        
    def forward(self, seq, seq_length):
        feats = seq[:, :, :-1]
        code = seq[:, :, -1].long()
        embed = self.type_embedding(code)
        feats_proj = self.proj(feats)
        x = torch.cat([feats_proj, embed], axis=-1)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        
#         x = x.permute(0, 2, 1)
#         x = self.conv1(x)
#         x = x.permute(0, 2, 1)
#         x = self.relu(x)
        
        x = rnn_utils.pack_padded_sequence(x, seq_length, batch_first=True, enforce_sorted=False)
        output, h_n = self.GRU(x)
        # output: [batch_size , seq_len, hidden_dim], h_n: [num_layers, batch_size, hidden_dim]
        #x = h_n.permute(1, 0, 2)[:, -1, :]
        out_pad, out_len = rnn_utils.pad_packed_sequence(output, batch_first=True)
        x = out_pad[torch.arange(out_len.shape[0]), out_len-1, :]
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        y = self.output(x)
        return x, y

In [17]:
class EmbeddingDistanceLoss(nn.Module):
    def __init__(self, num_classes, weight=None):
        super(EmbeddingDistanceLoss, self).__init__()
        self.num_classes = num_classes
        self.weight = weight
        
    def forward(self, output, label, embeddings):
        batch_size = output.shape[0]
        dist_mtrx = torch.cdist(output, embeddings, p=2)
        dist_mtrx = dist_mtrx*self.weight

        return (torch.mean(torch.tensor([-0.1/self.num_classes]).cuda()*dist_mtrx) + torch.mean(self.num_classes*dist_mtrx[:, label]))/batch_size

In [18]:
def update_class_embedding(output, label, class_embedding, alpha):
    tmp_embedding = torch.clone(class_embedding)
    tmp_embedding[label, :] = class_embedding[label, :]*(1-alpha) + output*alpha
    class_embedding = tmp_embedding

In [19]:
def metrices(preds, labels, num_classes):
    acc = []
    precision = []
    recall = []
    f1 = []
    count = [] 
    
    TP_all = 0
    FP_all = 0
    TN_all = 0
    FN_all = 0
    for target in range(num_classes):
        cnt = 0
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        for i in range(len(preds)):
            if labels[i]==target:
                cnt += 1
            if preds[i]==target and labels[i]==target:
                TP += 1
                TP_all += 1
            elif preds[i]==target and labels[i]!=target:
                FP += 1
                FP_all += 1
            elif preds[i]!=target and labels[i]!=target:
                TN += 1
                TN_all += 1
            elif preds[i]!=target and labels[i]==target:
                FN += 1
                FN_all += 1
        acc.append((TP+TN)/(TP+FP+TN+FN+1e-10))
        precision.append((TP)/(TP+FP+1e-10))
        recall.append(TP/(TP+FN+1e-10))
        f1.append(2*(precision[target]*recall[target])/(precision[target]+recall[target]+1e-10))
        count.append(cnt)
            
    num_exist_class = len([1 for c in count if c!=0]) + 1e-10
    return sum(acc)/num_exist_class, sum(precision)/num_exist_class, sum(recall)/num_exist_class, sum(f1)/num_exist_class, None

def confusion_matrix(preds, labels, num_classes, ax):
    conf = np.zeros((num_classes, num_classes))
    for i in range(len(preds)):
        conf[preds[i]][labels[i]] += 1

    norm_vec = np.sum(conf, axis=0)
    conf = np.around(conf/norm_vec, decimals=2)
    df_cm = pd.DataFrame(conf, [cat2player(i) for i in range(num_classes)], [cat2player(i) for i in range(num_classes)])
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 12}, cmap='GnBu', fmt='g', ax=ax)
    b, t = ax.get_ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    ax.set_title("Confusion Matrix of Player Classification")
    ax.set_ylim(b, t) # update the ylim(bottom, top) values
    ax.xaxis.set_tick_params(rotation=45)
    ax.yaxis.set_tick_params(rotation=45)
    ax.set_xlabel('Actual')
    ax.set_ylabel('Prediction')
    for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(12)
    ax.title.set_fontsize(16)


def validation(net, class_embedding, testloader, num_classes, epoch, ax, conf=False):
    labels = []
    preds = []
    for idx, (seq, seq_length, label) in enumerate(testloader):
        # seq: [batch x padded length x feat dim]
        # label [batch x 2]
        sigmoid = nn.Sigmoid()
        seq, label = torch.Tensor(seq).cuda(), torch.Tensor(label).long().cuda()
        output,_ = net(seq, seq_length)
        
        pred = torch.argmin(torch.cdist(output, class_embedding, p=2), dim=-1)
        #pred = torch.argmax(output, dim=-1)
        
        labels.append(label.cpu().item())
        preds.append(pred.detach().cpu().item())

    acc, prec, rec, f1, ACC = metrices(preds, labels, num_classes)
    if conf:
        confusion_matrix(preds, labels, num_classes, ax)
    #print("Epoch No.%d\tAcc: %3f, F1: %3f"%(epoch, rec, f1))
    return rec, f1

In [20]:
def moving_average(lst, wind_size):
    lst = [sum(lst[i:i+wind_size])/wind_size for i in range(0, len(lst)-wind_size)]
    return lst

def plots(losses, accs, f1s, wind_size, ax1):
    ax2 = ax1.twinx()
    
    y1 = moving_average(losses, wind_size)
    x1 = [i for i in range(len(y1))]
    
    y2 = accs
    x2_step = round(len(x1)/len(y2))
    x2 = [i*x2_step for i in range(1, len(y2)+1)]
    
    y3 = f1s

    curve1, = ax1.plot(x1, y1, label="Training loss", color='r')
    curve2, = ax2.plot(x2, y2, label="Test accuracy", color='b')
    curve3, = ax2.plot(x2, y3, label="Test F1", color='g')
    
    curves = [curve1, curve2, curve3]
    ax1.legend(curves, [curve.get_label() for curve in curves], loc='center right')
    ax1.set_xlabel("Iteration")
    ax1.set_ylabel("Loss")
    ax2.set_ylabel("Acc/F1")
    
    ax1.set_title("Learning Curve")
    
    for item in ([ax1.xaxis.label, ax1.yaxis.label, ax2.yaxis.label] + ax1.get_xticklabels() + ax1.get_yticklabels() +  ax2.get_yticklabels()):
        item.set_fontsize(12)
    ax1.title.set_fontsize(16)
    

In [21]:
def generate_class_embedding(num_classes, hidden_dim, model, pc_loader):
    with torch.no_grad():
        model.eval()
        class_embedding = torch.zeros(num_classes, hidden_dim).cuda()
        class_count = torch.zeros(num_classes).cuda()

        for seq, seq_length, labels in pc_loader:
            seq = seq.cuda()

            # labels: [batch_size]
            labels = labels.squeeze(1).long().cuda()
            # output: [batch_size x hidden_dim*2]
            output,_ = model(seq, seq_length)

            # add embeddings to corresponding class_embedding
            #scatter [num_classes x batch_size]
            scatter = torch.zeros(num_classes, labels.shape[0]).cuda()
            scatter[labels, torch.tensor([i for i in range(labels.shape[0])])] = 1

            # add class count
            class_count += scatter.sum(axis=1)

            # scatter*output [num_classes x hidden_dim*2]
            class_embedding += torch.matmul(scatter, output)

        class_embedding /= class_count.unsqueeze(1).expand(num_classes, hidden_dim)
    
    return class_embedding

In [22]:
def generate_all_embedding(num_classes, hidden_dim, model, pc_loader):
    with torch.no_grad():
        model.eval()
        embeddings = torch.zeros(0, hidden_dim).cuda()
        classes = torch.zeros(0).long().cuda()

        for seq, seq_length, labels in pc_loader:
            seq = seq.cuda()

            # labels: [batch_size]
            labels = labels.squeeze(1).long().cuda()
            # output: [batch_size x hidden_dim*2]
            output,_ = model(seq, seq_length)

            embeddings = torch.cat([embeddings, output], axis=0)
            classes = torch.cat([classes, labels], axis=0)
            
    return embeddings, classes.cpu().numpy()
    
def tsne_viz_train_test(train_embeddings, train_labels, test_embeddings, test_labels, class_embedding, ax, mode):
    train_range = train_embeddings.shape[0]
    test_range = train_range + test_embeddings.shape[0]
    embeddings = np.concatenate((train_embeddings, test_embeddings, class_embedding), axis=0)
    labels = np.concatenate((train_labels, test_labels, np.array([i for i in range(class_embedding.shape[0])])), axis=0)

    if mode=='tsne':
        reduced = TSNE(n_components=2, perplexity=30.0).fit_transform(embeddings)
    elif mode=='pca':
        reduced = PCA(n_components=2, svd_solver='full').fit_transform(embeddings)
    else:
        raise NotImplemented('Only t-SNE and PCA visualizations are supported currenly')
    
    train_x = reduced[:train_range, 0]
    train_y = reduced[:train_range, 1]
    train_group = labels[:train_range]
    
    test_x = reduced[train_range:test_range, 0]
    test_y = reduced[train_range:test_range, 1]
    test_group = labels[train_range:test_range]
    
    class_x = reduced[test_range:, 0]
    class_y = reduced[test_range:, 1]
    class_group = labels[test_range:]

    
    cdict = {0:'black', 1:'silver', 2:'lightcoral', 3:'red', 4:'sienna', 5:'orange',
           6:'yellow', 7:'green', 8:'lime', 9:'blue', 10:'cyan', 11:'purple',
           12:'lightblue', 13:'deeppink', 14:'darkmagenta', 15:'dodgerblue', 16:'lime'}

    for g in np.unique(train_group):
        ix = np.where(train_group == g)
        ax.scatter(train_x[ix], train_y[ix], c = cdict[g], s = 4, alpha=0.05)
    
    for g in np.unique(test_group):
        ix = np.where(test_group == g)
        ax.scatter(test_x[ix], test_y[ix], c = cdict[g], label = cat2player(g), s = 10, alpha=0.8)
    
    for g in np.unique(class_group):
        ix = np.where(class_group == g)
        ax.scatter(class_x[ix], class_y[ix], c = cdict[g], s = 200, marker = '*', edgecolors='black')
    
    ax.legend()
    if mode == 'tsne':
        ax.set_title('Visualization of Embeddings (t-SNE)')
    else:
        ax.set_title('Visualization of Embeddings (PCA)')
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])

In [23]:
train, test = train_test_split(rids, 0.2)

pc_dataset = PlayerClassificationDataset(data, train, 'train')
#pc_dataset = SubsequenceClassificationDataset(data, train, 'train', p=0.5, threshold=5)
weights = np.array([1, 1., 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
train_dataset = TripletSamplingDataset(pc_dataset, num_classes=11, weights=weights)
test_dataset = PlayerClassificationDataset(data, test, 'test')

Generating train labels: 100%|██████████| 1948/1948 [00:00<00:00, 2004.14it/s]


Removed 0 rallies with NaN value!


Generating test labels: 100%|██████████| 487/487 [00:00<00:00, 2076.90it/s]

Removed 0 rallies with NaN value!


In [24]:
pc_loader = DataLoader(pc_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=triplet_collate)
testloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [25]:
EPOCHS = 500
num_classes = 11
hidden_dim = 2048
lr = 1e-4

In [26]:
#net = SimpleNet(12, 128, hidden_dim, num_classes, 2).cuda()
net = CNNRNN(12, 128, hidden_dim, num_classes, 2).cuda()
class_embedding = torch.randn(num_classes, hidden_dim).cuda()
class_embedding.require_grad = False

optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-4)

triplet_loss = nn.TripletMarginLoss(margin=0.5, p=2).cuda()
softmax = nn.Softmax(dim=1)

In [27]:
train_embeddings, _ = generate_all_embedding(num_classes, hidden_dim, net, pc_loader)
class_embedding = generate_class_embedding(num_classes, hidden_dim, net, pc_loader)

trainloader.dataset.update_distance(train_embeddings)

In [ ]:
out = Output()
display.display(out)

losses = []
accs = [0]
f1s = [0]
gc.collect()

for epoch in tqdm(range(EPOCHS)):
    net.train()
    for idx, (anchor, positive, negative, anchor_len, positive_len, negative_len, anchor_label) in enumerate(trainloader):
        anchor, positive, negative = anchor.cuda(), positive.cuda(), negative.cuda()
        anchor_label = anchor_label.cuda()
        optimizer.zero_grad()
        anchor_embedding, acnhor_logits = net(anchor, anchor_len)
        positive_embedding, _ = net(positive, positive_len)
        negative_embedding, _ = net(negative, negative_len)
      
        t_loss = triplet_loss(anchor_embedding, positive_embedding, negative_embedding)

        loss = t_loss
        
        loss.backward()
        losses.append((loss).item())
        optimizer.step()

    if (epoch+1) % 25 == 0:
        train_embeddings, train_labels = generate_all_embedding(num_classes, hidden_dim, net, pc_loader)
        # update dist mtrx
        trainloader.dataset.update_distance(train_embeddings)
        test_embeddings, test_labels = generate_all_embedding(num_classes, hidden_dim, net, testloader)
        class_embedding = generate_class_embedding(num_classes, hidden_dim, net, pc_loader)
        with out:
            #clear_output(wait=True)
            fig, axes = plt.subplots(1, 2, figsize=(25, 10))
            acc, f1 = validation(net, class_embedding, testloader, num_classes, epoch, axes[0], conf=True)
            accs.append(acc)
            f1s.append(f1)
            plots(losses, accs, f1s, 10, axes[1])
            plt.show()
            
            fig, axes = plt.subplots(1, 2, figsize=(25, 10))
            
            tsne_viz_train_test(train_embeddings.cpu().numpy(), train_labels, test_embeddings.cpu().numpy(), test_labels, class_embedding.cpu(), axes[0], 'pca')
            tsne_viz_train_test(train_embeddings.cpu().numpy(), train_labels, test_embeddings.cpu().numpy(), test_labels, class_embedding.cpu(), axes[1], 'tsne')
            plt.show()

    else:
        train_embeddings, train_labels = generate_all_embedding(num_classes, hidden_dim, net, pc_loader)
        # update dist mtrx every epoch
        trainloader.dataset.update_distance(train_embeddings)
        class_embedding = generate_class_embedding(num_classes, hidden_dim, net, pc_loader)
        acc, f1 = validation(net, class_embedding, testloader, num_classes, epoch, ax=None)
        accs.append(acc)
        f1s.append(f1)

Output()

  2%|▏         | 10/500 [13:48<11:26:14, 84.03s/it]

In [ ]:
torch.save(net.state_dict(), 'for_5_29_no_girl_re.pt')

In [ ]:
def class_distance_viz(class_embedding):
    class_dist = torch.cdist(class_embedding, class_embedding, p=2).cpu().numpy()#/class_embedding.shape[1]
    fig, ax = plt.subplots(figsize=(12, 10))
    df = pd.DataFrame(class_dist, [cat2player(i) for i in range(num_classes)], [cat2player(i) for i in range(num_classes)])
    
    sn.heatmap(df, annot=True, annot_kws={"size": 10}, cmap='GnBu', fmt='.2f', ax=ax, square=True)
    b, t = ax.get_ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    ax.set_title("Distance of Embedding between Players")
    ax.set_ylim(b, t) # update the ylim(bottom, top) values
    ax.xaxis.set_tick_params(rotation=90)
    ax.yaxis.set_tick_params(rotation=0)
    ax.set_xlabel('Player')
    ax.set_ylabel('Player')
    for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(12)
    ax.title.set_fontsize(16)

In [ ]:
class_distance_viz(class_embedding)

In [ ]:
accs[-1]

In [ ]:
f1s[-1]

In [ ]:
embedding = class_embedding.clone().cpu().numpy()

In [ ]:
p2c = {'Anders ANTONSEN': 0, 'Anthony Sinisuka GINTING': 1, 'CHEN Long': 2,
      'CHOU Tien Chen': 3, 'Jonatan CHRISTIE': 4, 'Kento MOMOTA': 5,
     'Khosit PHETPRADAB': 6, 'NG Ka Long Angus': 7,
     'SHI Yuqi': 8,  'Viktor AXELSEN': 9, 'WANG Tzu Wei': 10}

In [ ]:
output = dict(zip(list(p2c.keys()), embedding))

In [ ]:
import pickle

In [ ]:
with open('5_29_player_embedding_no_girl_re', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 5/29